In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("create_merchant_recommendation_dataMart").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [3]:
# CRITICAL FIXES: Must be run BEFORE loading data to prevent ClassCastException
# 1. Disable Vectorized Reader (Avoids low-level ORC data reading crash)
spark.conf.set("spark.sql.orc.enableVectorizedReader", "false")
spark.conf.set("spark.sql.hive.convertMetastoreOrc", "false")
# 2. Disable Broadcast Join (Avoids memory/shuffle crash on join)
#spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1) 
# 3. Disable Spark's optimizing components (Forces safer execution path)
#spark.conf.set("spark.sql.cbo.enabled", "false") 
#spark.conf.set("spark.sql.codegen.wholeStage", "false")
# 4. Force Hive SerDe (Ultimate attempt to bypass native Spark reader)
spark.conf.set("spark.sql.hive.convertMetastore", "false") 

print("Spark Config Updated for maximum stability.")



Spark Config Updated for maximum stability.


In [4]:
import pandas as pd
import numpy as np
import pyspark.sql.functions as F 
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType, DecimalType, TimestampType

In [5]:
#Load the tables directly from the Database
#We use the standard "database_name.table_name" format

print("Loading Silver Layer table from Hive..")

#A. Transaction Table
df_trans=spark.table("financial_db.transactions_silver")

#B. Users Table
df_users=spark.table("financial_db.users_silver")

#C.Cards Table
df_cards=spark.table("financial_db.cards_silver")

#3.Verification
print(f"Transaction Count:{df_trans.count()}")
print(f"Users Count: {df_users.count()}")
print(f"Cards Count: {df_cards.count()}")

#4.Preview Schema to ensure types are correct
df_trans.printSchema()
df_users.printSchema()
df_cards.printSchema()

Loading Silver Layer table from Hive..
Transaction Count:24386900
Users Count: 2000
Cards Count: 6146
root
 |-- user_id: integer (nullable = true)
 |-- card_id: integer (nullable = true)
 |-- amount: decimal(10,2) (nullable = true)
 |-- use_chip: string (nullable = true)
 |-- merchant_name: string (nullable = true)
 |-- merchant_city: string (nullable = true)
 |-- merchant_state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- mcc: integer (nullable = true)
 |-- errors: string (nullable = true)
 |-- is_fraud: string (nullable = true)
 |-- transaction_timestamp: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)

root
 |-- person_id: string (nullable = true)
 |-- current_age: integer (nullable = true)
 |-- retirement_age: integer (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- birth_month: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- address: string (nullable = true)
 |-- apart

In [6]:
df_trans.show()

+-------+-------+------+--------+--------------------+--------------+--------------+-----+----+------+--------+---------------------+----+-----+
|user_id|card_id|amount|use_chip|       merchant_name| merchant_city|merchant_state|  zip| mcc|errors|is_fraud|transaction_timestamp|year|month|
+-------+-------+------+--------+--------------------+--------------+--------------+-----+----+------+--------+---------------------+----+-----+
|    928|      2| 86.64|   Swipe|-8728837507959629735|  Chesterfield|            VA|23832|4900|   N/A|      No|  2017-09-02 13:40:00|2017|    9|
|   1762|      0| 99.23|   Swipe| 6568396818790153121|     Blackwell|            OK|74631|4900|   N/A|      No|  2017-09-01 07:00:00|2017|    9|
|    928|      2|140.00|   Swipe|-4282466774399734331|    South Hill|            VA|23970|4829|   N/A|      No|  2017-09-03 00:15:00|2017|    9|
|   1762|      0| 43.99|   Swipe|-2555314073116318866|     Blackwell|            OK|74631|5651|   N/A|      No|  2017-09-01 20:43:

In [7]:
df_cards.select("card_number","card_type").show(10)

+----------------+---------------+
|     card_number|      card_type|
+----------------+---------------+
|4344676511950444|          Debit|
|4956965974959986|          Debit|
|4582313478255491|          Debit|
|4879494103069057|         Credit|
|5722874738736011|Debit (Prepaid)|
|4404898874682993|         Credit|
|4001482973848631|          Debit|
|5627220683410948|          Debit|
|5711382187309326|Debit (Prepaid)|
|5766121508358701|Debit (Prepaid)|
+----------------+---------------+
only showing top 10 rows



In [8]:
print("----2.Fixing Users Table----")

#1. Generate 'user_id' using Row Number(0,1,2)
rdd_with_id=df_users.rdd.zipWithIndex().map(lambda x:(x[1],)+tuple(x[0]))

#2.Create DataFrame with "user_id" as the first column 
new_column_names=["user_id"]+df_users.columns

df_users_indexed=spark.createDataFrame(rdd_with_id,new_column_names)


----2.Fixing Users Table----


In [9]:
df_users_indexed.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- person_id: string (nullable = true)
 |-- current_age: long (nullable = true)
 |-- retirement_age: long (nullable = true)
 |-- birth_year: long (nullable = true)
 |-- birth_month: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- address: string (nullable = true)
 |-- apartment: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- per_capita_income_zipcode: decimal(38,18) (nullable = true)
 |-- yearly_income_person: decimal(38,18) (nullable = true)
 |-- total_debt: decimal(38,18) (nullable = true)
 |-- fico_score: long (nullable = true)
 |-- num_credit_cards: long (nullable = true)



In [10]:
#Dropping the is_fraud col and removing the rows where is_froud = 'No'
df_trans_prep=df_trans.filter(F.col("errors")=='N/A').filter(F.col("amount")>0).filter(F.col("is_fraud")=="No")
df_trans_prep=df_trans_prep.drop("is_fraud","errors","use_chip")
print("---printing the final schema of the prepped transaction table---")
df_trans_prep.printSchema()


#selecting the necessary fields from the df_cards
df_users_prep=df_users_indexed.select(
    F.col("user_id"),
    F.col("person_id").alias("person_name"),
    F.col("current_age"),
    F.col("gender"),
    F.col("city"),
    F.col("zipcode"),F.col("fico_score"),F.col("num_credit_cards"))
print("---schema of the final columns selected from the users table---")
df_users_prep.printSchema()


#selecting the necessary fields from the df_cards
df_cards_prep=df_cards.select(
    F.col("user").alias("user_id"),
    F.col("card_index").alias("card_id"),
    F.col("card_brand"),
    F.col("card_type"),
    F.col("credit_limit"))
print("---schema of the final columns selected from the cards table---")
df_cards_prep.printSchema()

---printing the final schema of the prepped transaction table---
root
 |-- user_id: integer (nullable = true)
 |-- card_id: integer (nullable = true)
 |-- amount: decimal(10,2) (nullable = true)
 |-- merchant_name: string (nullable = true)
 |-- merchant_city: string (nullable = true)
 |-- merchant_state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- mcc: integer (nullable = true)
 |-- transaction_timestamp: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)

---schema of the final columns selected from the users table---
root
 |-- user_id: long (nullable = true)
 |-- person_name: string (nullable = true)
 |-- current_age: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- city: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- fico_score: long (nullable = true)
 |-- num_credit_cards: long (nullable = true)

---schema of the final columns selected from the cards table---
root
 |-- us

In [11]:
df_users_prep.show(5)

+-------+--------------+-----------+------+-------------+-------+----------+----------------+
|user_id|   person_name|current_age|gender|         city|zipcode|fico_score|num_credit_cards|
+-------+--------------+-----------+------+-------------+-------+----------+----------------+
|      0|Hazel Robinson|         53|Female|     La Verne|  91750|       787|               5|
|      1|    Sasha Sadr|         53|Female|  Little Neck|  11363|       701|               5|
|      2|    Saanvi Lee|         81|Female|  West Covina|  91792|       698|               5|
|      3| Everlee Clark|         63|Female|     New York|  10069|       722|               4|
|      4| Kyle Peterson|         43|  Male|San Francisco|  94117|       675|               1|
+-------+--------------+-----------+------+-------------+-------+----------+----------------+
only showing top 5 rows



In [12]:
df_trans_prep.show(10)

+-------+-------+------+--------------------+--------------+--------------+-----+----+---------------------+----+-----+
|user_id|card_id|amount|       merchant_name| merchant_city|merchant_state|  zip| mcc|transaction_timestamp|year|month|
+-------+-------+------+--------------------+--------------+--------------+-----+----+---------------------+----+-----+
|    235|      1| 64.00| 1799189980464955940|       Garland|            TX|75042|5499|  2018-11-01 10:10:00|2018|   11|
|    591|      3|117.97|  593588911319431962|        Tucker|            GA|30084|5411|  2018-11-01 14:58:00|2018|   11|
|    235|      1| 41.10| 1799189980464955940|       Garland|            TX|75042|5499|  2018-11-01 10:10:00|2018|   11|
|    591|      3| 10.55| 6935085019857174168|        Tucker|            GA|30084|5912|  2018-11-05 10:34:00|2018|   11|
|    591|      3| 12.52| 6135208568923449408|        Tucker|            GA|30084|9402|  2018-11-06 16:33:00|2018|   11|
|    591|      3| 69.31| 341452745957910

In [13]:
df_cards_prep.show(5)

+-------+-------+----------+---------------+------------+
|user_id|card_id|card_brand|      card_type|credit_limit|
+-------+-------+----------+---------------+------------+
|      0|      0|      Visa|          Debit|    24295.00|
|      0|      1|      Visa|          Debit|    21968.00|
|      0|      2|      Visa|          Debit|    46414.00|
|      0|      3|      Visa|         Credit|    12400.00|
|      0|      4|Mastercard|Debit (Prepaid)|       28.00|
+-------+-------+----------+---------------+------------+
only showing top 5 rows



In [14]:
df_join=df_trans_prep.join(df_users_prep,on="user_id")

#joining the cards table with df_join
df_join_final = df_join.join(df_cards_prep, 
                             (df_join.user_id == df_cards_prep.user_id) & 
                             (df_join.card_id == df_cards_prep.card_index), 
                             how="left").drop(df_cards_prep.user_id)

In [15]:
df_join_final = df_join.fillna(0, subset=["card_id"]).join(
    df_cards_prep,
    on=["user_id", "card_id"],
    how="left"
)

In [16]:
df_join_final.show(5)

+-------+-------+------+--------------------+-------------+--------------+-----+----+---------------------+----+-----+-------------+-----------+------+-------+-------+----------+----------------+----------+---------+------------+
|user_id|card_id|amount|       merchant_name|merchant_city|merchant_state|  zip| mcc|transaction_timestamp|year|month|  person_name|current_age|gender|   city|zipcode|fico_score|num_credit_cards|card_brand|card_type|credit_limit|
+-------+-------+------+--------------------+-------------+--------------+-----+----+---------------------+----+-----+-------------+-----------+------+-------+-------+----------+----------------+----------+---------+------------+
|     26|      0|  2.24|-7899746529920728234|        Miami|            FL|33196|5411|  2017-03-01 07:07:00|2017|    3|Amaya Stewart|         40|Female|Houston|  77061|       722|               2|Mastercard|    Debit|    13601.00|
|     26|      0|  9.22|-5467922351692495955|        Miami|            FL|33196|

In [17]:
df_join_final.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- card_id: integer (nullable = true)
 |-- amount: decimal(10,2) (nullable = true)
 |-- merchant_name: string (nullable = true)
 |-- merchant_city: string (nullable = true)
 |-- merchant_state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- mcc: integer (nullable = true)
 |-- transaction_timestamp: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- person_name: string (nullable = true)
 |-- current_age: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- city: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- fico_score: long (nullable = true)
 |-- num_credit_cards: long (nullable = true)
 |-- card_brand: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- credit_limit: decimal(10,2) (nullable = true)



In [22]:

from pyspark.sql.window import Window
df_mart = df_join_final.withColumn(
    # 1. Composite Transaction ID
    "transaction_id",
    F.concat_ws("-", F.col("user_id"), F.col("card_id"), F.col("transaction_timestamp").cast("long"))
).withColumn("is_weekend", F.dayofweek("transaction_timestamp").isin(1, 7)) \
    .withColumn("day_part", 
        F.when((F.hour("transaction_timestamp") >= 5) & (F.hour("transaction_timestamp") < 12), "Morning")
        .when((F.hour("transaction_timestamp") >= 12) & (F.hour("transaction_timestamp") < 17), "Afternoon")
        .when((F.hour("transaction_timestamp") >= 17) & (F.hour("transaction_timestamp") < 21), "Evening")
        .otherwise("Night")) \
    .withColumn("utilization_rate", F.col("amount") / F.col("credit_limit")) \
    .withColumn("is_local", F.col("city") == F.col("merchant_city"))




# Adding  Loyalty Signal (How many times has this user been here before?)
#window_spec = Window.partitionBy("user_id", "merchant_name").orderBy("transaction_timestamp")
#df_mart = df_mart.withColumn("cumulative_visit_count", F.row_number().over(window_spec))

In [23]:
df_mart=df_mart.withColumn("merchant_zip",F.col("zip"))
df_mart=df_mart.drop("fico_score","num_credit_cards","card_index")


In [24]:
df_mart=df_mart.drop("zip")

In [25]:
print("--- Step 6: Saving as External Hive Table ---")
# Path for the External Table
gold_path = "/user/talentum/projectMaster/warehouseDir/gold/merchants_recommendation"
spark.sql("DROP TABLE IF EXISTS financial_db.merchants_recommendation_gold")

# Write with explicit partitioning and Hive serde override
df_mart.write \
    .mode("overwrite") \
    .format("orc") \
    .option("path", "/user/talentum/projectMaster/warehouseDir/gold/merchants_recommendation") \
    .option("compression", "snappy") \
    .saveAsTable("financial_db.merchants_recommendation_gold")

print("SUCCESS: Table saved!")

--- Step 6: Saving as External Hive Table ---
SUCCESS: Table saved!
